In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from vamos import optimize, OptimizeConfig, NSGAIIConfig
from vamos.foundation.problem import Problem
from vamos.foundation.problem.constrained import WeldedBeamDesignProblem, CEC2009CF1

plt.style.use("ggplot")
print("Constrained optimization tools loaded!")

## 1. Constraint Handling in VAMOS

VAMOS problems can return a constraint matrix `G` (shape: `[n_points, n_constraints]`).
- `G[i, j] <= 0` → constraint j satisfied
- `G[i, j] > 0` → constraint j violated (violation amount = G[i, j])

Problems store constraints in `out["G"]` with shape `(n_points, n_constraints)`.

In [ ]:
# Example: manual constraint evaluation
G_example = np.array([
    [-0.5, -0.1],  # Feasible (both <= 0)
    [0.1, -0.2],   # Infeasible (first > 0)
    [-0.3, 0.5],   # Infeasible (second > 0)
    [0.0, 0.0],    # Feasible (boundary)
])

from vamos.foundation.metrics.const_handling import ConstraintInfo

# ConstraintInfo helps analyze feasibility
info = ConstraintInfo(G_example)

print(f"Constraint violation (CV) per solution: {info.cv}")
print(f"Feasible mask: {info.feasible_mask}")
print(f"Number of feasible solutions: {info.feasible_mask.sum()} / {len(info.feasible_mask)}")

## 2. Constraint Handling Strategies

VAMOS supports multiple strategies for handling constraints during selection:

| Strategy | Description |
| :--- | :--- |
| `FeasibilityFirstStrategy` | (Default) Always prefers feasible solutions over infeasible ones. Among infeasible, minimizes CV. |
| `EpsilonConstraintStrategy` | Feasibility-first with epsilon tolerance |
| `CVAsObjectiveStrategy` | Treats CV as a ranking criterion with objective tie-break |

In [ ]:
# Example: Compare strategies
F = np.array([
    [1.0, 2.0],   # Good objectives, feasible
    [0.5, 0.5],   # Excellent objectives, but INFEASIBLE (CV=1.0)
    [2.0, 3.0],   # Poor objectives, feasible
    [0.6, 0.6],   # Good objectives, slightly infeasible (CV=0.1)
])
G = np.array([
    [-1.0], [1.0], [-1.0], [0.1] 
])

from vamos.engine.selection.strategies import FeasibilityFirstStrategy, EpsilonConstraintStrategy

strategies = {
    "FeasibilityFirst": FeasibilityFirstStrategy(),
    "Epsilon(0.15)": EpsilonConstraintStrategy(epsilon=0.15)
}

print("Ranking (lower rank is better):")
for name, strategy in strategies.items():
    # Note: strategies return ranks/indices
    # This is a conceptual demonstration
    cinfo = ConstraintInfo(G)
    # Mock ranking (in real usage, this is integrated into sorting)
    # Here we just show what they prioritize
    print(f"  {name}:")
    if name == "FeasibilityFirst":
        print("    1. [1.0, 2.0] (Feasible)")
        print("    2. [2.0, 3.0] (Feasible)")
        print("    3. [0.6, 0.6] (CV=0.1)")
        print("    4. [0.5, 0.5] (CV=1.0)")
    else:
        print("    1. [0.6, 0.6] (CV=0.1 <= 0.15, treated as feasible, better objs)")
        print("    2. [1.0, 2.0] (Feasible)")
        print("    3. [2.0, 3.0] (Feasible)")
        print("    4. [0.5, 0.5] (CV=1.0)")

print()
print("Note: FeasibilityFirst penalizes infeasible solutions heavily.")
print("      Epsilon with ε=0.15 treats solution 4 (CV=0.1) as feasible.")

## 3. Constrained Engineering Problem: Welded Beam

The welded beam design problem has:
- **6 variables**: weld thickness, length, flange thickness, width, material choice, stiffener count
- **2 objectives**: minimize cost, minimize deflection
- **4 constraints**: shear stress, normal stress, deflection limit, geometric coupling

In [ ]:
# Create welded beam problem
welded_beam = WeldedBeamDesignProblem()

print(f"Welded Beam Design:")
print(f"  {welded_beam.n_var} variables")
print(f"  {welded_beam.n_obj} objectives")
print(f"  {welded_beam.n_constr} constraints")
print(f"  Bounds: {list(zip(welded_beam.xl, welded_beam.xu))}")
print()
print("Constraints (G <= 0 = feasible):")
print("  g1: shear stress <= limit")
print("  g2: normal stress <= limit")
print("  g3: deflection <= 0.25")
print("  g4: h <= b (geometric)")

In [ ]:
# Configure algorithm for constrained problem
constrained_config = (
    NSGAIIConfig()
    .pop_size(100)
    .engine("numpy")
    .fixed()
)
# Note: VAMOS algorithms adapt standard Non-Dominated Sort to 
# "Constrained Domination Sort" automatically if the problem returns constraints.

print("Running constrained optimization...")
welded_result = optimize(OptimizeConfig(
    problem=welded_beam,
    algorithm="nsgaii",
    algorithm_config=constrained_config,
    termination=("n_eval", 10000),
    seed=42,
))

print(f"Found {len(welded_result.F)} solutions")

In [ ]:
# Analyze feasibility of final population
if welded_result.G is not None:
    cinfo = ConstraintInfo(welded_result.G)
    print(f"Final Population Feasibility:")
    print(f"  Feasible: {cinfo.feasible_mask.sum()} / {len(cinfo.feasible_mask)}")
    print(f"  Infeasible: {(~cinfo.feasible_mask).sum()}")
    if not cinfo.feasible_mask.all():
        print(f"  Average CV (infeasible only): {cinfo.cv[~cinfo.feasible_mask].mean():.4f}")
else:
    print("No constraint data returned.")
    cinfo = None

In [ ]:
# Visualize: color by feasibility
fig, ax = plt.subplots(figsize=(8, 6))

F = welded_result.F
mask = cinfo.feasible_mask if cinfo else np.ones(len(F), dtype=bool)

# Feasible solutions
ax.scatter(F[mask, 0], F[mask, 1], c='green', label='Feasible', alpha=0.7)

# Infeasible solutions
if (~mask).any():
    ax.scatter(F[~mask, 0], F[~mask, 1], c='red', marker='x', label='Infeasible', alpha=0.5)

ax.set_xlabel("Cost")
ax.set_ylabel("Deflection")
ax.set_title("Welded Beam: Cost vs Deflection (Constrained)")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 4. CEC2009 Constrained Test Function (CF1)

The CEC2009 CF suite provides challenging constrained multi-objective problems.

In [ ]:
cf1 = CEC2009CF1(n_var=10)

print(f"CF1 Problem:")
print(f"  {cf1.n_var} variables")
print(f"  {cf1.n_obj} objectives")
print(f"  {cf1.n_constr} constraints")
print(f"  Bounds: [{cf1.xl[0]:.1f}, {cf1.xu[0]:.1f}]")
print()
print("The CF1 constraint creates infeasible bands across the Pareto front,")
print("making parts of the objective space unreachable.")

In [ ]:
# Run on CF1
cf1_config = (
    NSGAIIConfig()
    .pop_size(100)
    .engine("numpy")
    .fixed()
)

print("Running on CF1 (this may take a moment)...")
cf1_result = optimize(OptimizeConfig(
    problem=cf1,
    algorithm="nsgaii",
    algorithm_config=cf1_config,
    termination=("n_eval", 5000),
    seed=42,
))

# Check feasibility
if cf1_result.G is not None:
    cf1_cinfo = ConstraintInfo(cf1_result.G)
    n_feas = cf1_cinfo.feasible_mask.sum()
    print(f"Feasible: {n_feas} / {len(cf1_cinfo.feasible_mask)}")
else:
    cf1_cinfo = None
    print("No constraint data.")

In [ ]:
# Visualize CF1
fig, ax = plt.subplots(figsize=(8, 6))

F = cf1_result.F
mask = cf1_cinfo.feasible_mask

ax.scatter(F[mask, 0], F[mask, 1], c='green', label='Feasible', alpha=0.6)
ax.scatter(F[~mask, 0], F[~mask, 1], c='red', marker='x', label='Infeasible', alpha=0.3)

ax.set_title(f"CEC2009 CF1\n({mask.sum()} feasible / {len(F)} total)")
ax.set_xlabel("f1")
ax.set_ylabel("f2")
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.show()

## 5. Alternative: Manual Penalty Approach

Sometimes you may want to handle constraints manually by adding a penalty to the objectives. This is useful when:
- You want "soft" constraints (allow violation if benefit is high enough).
- You want to control the penalty severity explicitly.
- You are using an algorithm that doesn't support constraint handling natively.

**Penalty Function:**
$$f_i'(x) = f_i(x) + P \cdot CV(x)$$

Where $P$ is a penalty coefficient (e.g., 1000).

In [ ]:
class PenalizedWeldedBeam(WeldedBeamDesignProblem):
    def __init__(self, penalty_coeff=1000.0):
        super().__init__()
        self.penalty_coeff = penalty_coeff
        # HACK: Tell VAMOS we have 0 constraints so it doesn't do constrained domination
        self.n_constr = 0
    
    def evaluate(self, X, out, *args, **kwargs):
        # 1. Call original evaluate to get F and G
        # We'll use a temporary dictionary to capture the output
        temp_out = {}
        super().evaluate(X, temp_out, *args, **kwargs)
        
        F_orig = temp_out["F"]
        G_orig = temp_out["G"]
        
        # 2. Calculate CV
        # CV = sum of positive G values
        CV = np.sum(np.maximum(0, G_orig), axis=1)
        
        # 3. Add penalty to objectives
        # Reshape CV to match F shape for broadcasting if needed, or just add to columns
        F_penalized = F_orig + self.penalty_coeff * CV[:, None]
        
        # 4. Return penalized objectives
        out["F"] = F_penalized
        # We do NOT return G, so the algorithm sees it as unconstrained

# Run with manual penalty
print("Running with Manual Penalty (P=1000)...")
penalized_problem = PenalizedWeldedBeam(penalty_coeff=1000.0)

penalized_result = optimize(OptimizeConfig(
    problem=penalized_problem,
    algorithm="nsgaii",
    algorithm_config=NSGAIIConfig().pop_size(100).engine("numpy").fixed(),
    termination=("n_eval", 10000),
    seed=42,
))

# IMPORTANT: We need to re-evaluate the true objectives and constraints
# to see if the solutions are actually feasible.
print("Verifying feasibility of penalized results...")
true_problem = WeldedBeamDesignProblem()
verification_out = {}
true_problem.evaluate(penalized_result.X, verification_out)

final_cinfo = ConstraintInfo(verification_out["G"])
n_feas = final_cinfo.feasible_mask.sum()
print(f"Feasible solutions found: {n_feas} / {len(penalized_result.X)}")

# Plot comparison
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Plot 1: Standard Constrained Handling
F1 = welded_result.F
c1 = ConstraintInfo(welded_result.G)
axes[0].scatter(F1[c1.feasible_mask, 0], F1[c1.feasible_mask, 1], c='green', label='Feasible')
axes[0].scatter(F1[~c1.feasible_mask, 0], F1[~c1.feasible_mask, 1], c='red', marker='x', label='Infeasible')
axes[0].set_title("Method A: Constrained Domination (Standard)")
axes[0].set_xlabel("Cost")
axes[0].set_ylabel("Deflection")

# Plot 2: Manual Penalty
F2 = verification_out["F"] # True objectives, not penalized ones
c2 = ConstraintInfo(verification_out["G"])
axes[1].scatter(F2[c2.feasible_mask, 0], F2[c2.feasible_mask, 1], c='green', label='Feasible')
axes[1].scatter(F2[~c2.feasible_mask, 0], F2[~c2.feasible_mask, 1], c='red', marker='x', label='Infeasible')
axes[1].set_title("Method B: Manual Penalty (P=1000)")
axes[1].set_xlabel("Cost")

plt.tight_layout()
plt.show()

## Summary

| Method | Pros | Cons |
| :--- | :--- | :--- |
| **Constrained Domination** (Standard) | No tuning required. Guaranteed to prefer feasible solutions. | May get stuck if feasible region is hard to find. |
| **Manual Penalty** | Can create gradients through infeasible space. Allows soft constraints. | Requires tuning 'P'. Results depend heavily on penalty magnitude. |
